In [13]:
import numpy as np
import pescador
import logging
import os

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
LOGGER = logging.getLogger('gbsd')
LOGGER.setLevel(logging.DEBUG)

In [15]:
matplotlib.use('Agg')

In [16]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [17]:
CMD_VOLENVPER = 0
CMD_DUTYLL = 1
CMD_MSB = 2
CMD_LSB = 3
CMD_COUNT = 4

def onehot_cmd(data):
    cmd = data[CMD_OFFSET]
    nd = [ 0, 0, 0, 0 ]
    nd[int(cmd)] = 1
    return nd

TIME_OFFSET = 0
CH_1_OFFSET = 1
CH_2_OFFSET = 2
CMD_OFFSET = 3
CHANNEL_OFFSET = 4
DUTY_OFFSET = 5
LENGTH_OFFSET = 6
VOL_OFFSET = 7
ADD_MODE_OFFSET = 8
PERIOD_OFFSET = 9
FREQLSB_OFFSET = 10
FREQMSB_OFFSET = 11
LENGTH_ENABLE_OFFSET = 12
TRIGGER_OFFSET = 13
NUM_INP = 14

WINDOW_SIZE = 1024

NORMALIZE_TIME_BY = float(4194304 * 3) # 1 second is 4194304 cycles so this is 10s

def norm(val, max_val):
    return ((val / max_val) * 2.) - 1.

def unnorm(val, max_val):
    return ((val + 1.) / 2.) * max_val

def fresh_input(command, channel, time):
    newd = np.zeros(shape=NUM_INP, dtype=float)
    newd[TIME_OFFSET] = norm(time, NORMALIZE_TIME_BY)
    #print(channel)
    if int(channel) == 1:
        newd[CH_1_OFFSET] = 1.
    elif int(channel) == 2:
        newd[CH_2_OFFSET] = 1.
    else:
        raise "I didn't expect this"
    newd[CMD_OFFSET] = channel
    return newd

def nop():
    return fresh_input(NOP_CMD_OFFSET, 1, 0)

def duty_ll(channel, parts, time):
    inp = fresh_input(CMD_DUTYLL, channel, time)
    inp[DUTY_OFFSET] = norm(float(parts[3]), 2.)
    inp[LENGTH_OFFSET] = norm(float(parts[4]), 64.)
    return inp

def volenvper(channel, parts, time):
    inp = fresh_input(CMD_VOLENVPER, channel, time)
    inp[VOL_OFFSET] = float(parts[3]) / 16.
    inp[ADD_MODE_OFFSET] = float(parts[4])
    inp[PERIOD_OFFSET] = float(parts[4]) / 7.
    return inp

def freqlsb(channel, parts, time):
    inp = fresh_input(CMD_LSB, channel, time)
    inp[FREQLSB_OFFSET] = norm(float(parts[3]), 255.)
    return inp

def freqmsb(channel, parts, time):
    inp = fresh_input(CMD_MSB, channel, time)
    inp[FREQMSB_OFFSET] = norm(float(parts[3]), 7.)
    inp[LENGTH_ENABLE_OFFSET] = float(bool(parts[4]))
    inp[TRIGGER_OFFSET] = float(bool(parts[5]))
    return inp

def load_training_data(src):
    data = []
    file = open(src, 'r')
    for line in file:
        parts = line.split()
        if len(parts) > 0 and parts[0] == "CH":
           #print(parts)
           channel = int(parts[1])
           command = parts[2]
           time = int(parts[-1])
           if command == "DUTYLL":
               data.append(duty_ll(channel, parts, time))
           if command == "VOLENVPER":
               data.append(volenvper(channel, parts, time))
           if command == "FREQLSB":
               data.append(freqlsb(channel, parts, time))
           if command == "FREQMSB":
               data.append(freqmsb(channel, parts, time))
           #print("NEXTCMD", data[-1])
    return data

@pescador.streamable
def samples_from_training_data(src, window_size=WINDOW_SIZE):
    sample_data = None

    try:
        sample_data = load_training_data(src)
    except Exception as e:
        LOGGER.error('Could not load {}: {}'.format(src, str(e)))
        raise StopIteration()

    true_window_size = window_size + 1

    # Pad small samples with nop
    while len(sample_data) < true_window_size:
        sample_data.append(nop())

    while True:

        if len(sample_data) == true_window_size:
            sample = sample_data
        else:
            # Sample a random window from the audio file
            start_idx = np.random.randint(0, len(sample_data) - true_window_size)
            end_idx = start_idx + true_window_size
            sample = sample_data[start_idx:end_idx]

        sample_input = sample[0:window_size]
        sample_output = sample[window_size:window_size+1]

        sample_input = np.array(sample_input).astype(np.float32)
        sample_output = np.array(sample_output).astype(np.float32)

        yield { 'X':sample_input, 'Y': sample_output }

def create_batch_generator(paths, batch_size):
    streamers = []
    for path in paths:
        print("Creating a batch generator")
        streamers.append(samples_from_training_data(path))
        print("Done creating batch generator")
    mux = pescador.ShuffledMux(streamers)
    batch_gen = pescador.buffer_stream(mux, batch_size)
    return batch_gen

def training_files(dirp):
    return [
      os.path.join(root, fname)
      for (root, dir_names, file_names) in os.walk(dirp, followlinks=True)
      for fname in file_names
    ]

def create_data_split(paths, batch_size):
    train_gen = create_batch_generator(paths, batch_size)
    return train_gen



In [18]:
EPOCHS = 20
ROUND_SZ = 1000

In [19]:
NUM_EVENTS_PER_ROUND = WINDOW_SIZE
DIM = NUM_INP * NUM_EVENTS_PER_ROUND

class TimeNet(nn.Module):
    def __init__(self):
        super(TimeNet, self).__init__()

        self.main = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(WINDOW_SIZE, 64),
            nn.Sigmoid(),
            nn.Linear(64, 1)
        )

    def forward(self, noise):
        output = self.main(noise)
        return output

class CommandNet(nn.Module):

    def __init__(self):
        super(CommandNet, self).__init__()

        self.cmd_embedding = nn.Embedding(CMD_COUNT, 1)

        self.main = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.6),
            nn.Conv1d(1, 1, kernel_size=NUM_INP),
            nn.Conv1d(1, 1, kernel_size=NUM_INP),
            nn.Conv1d(1, 1, kernel_size=NUM_INP),
            nn.ReLU(),
            nn.Linear(985, 128),
            nn.Dropout(p=0.2),
            nn.Linear(128, CMD_COUNT),
            nn.Softmax(dim=1)
        )

    def forward(self, sequence):
        sequence = [self.cmd_embedding(sequence[:,i + CMD_OFFSET].type(torch.IntTensor).to(device)) for i in range(0, DIM, NUM_INP)]
        sequence = torch.cat(sequence, 1)
        output = self.main(sequence)
        return output

class ChannelNet(nn.Module):

    def __init__(self):
        super(ChannelNet, self).__init__()

        self.main = nn.Sequential(
            nn.Flatt
            nn.Dropout(p=0.6),
            nn.Linear(DIM, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Sigmoid(),
            nn.Linear(512, 2),

            nn.Softmax(dim=1)
        )

    def forward(self, noise):
        output = self.main(noise)
        return output

class FreqNet(nn.Module):

    def __init__(self):
        super(FreqNet, self).__init__()

        self.main = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(WINDOW_SIZE, 64),
            nn.Sigmoid(),
            nn.Linear(64, 1)
        )

    def forward(self, noise):
        output = self.main(noise)
        return output


In [25]:
def extract_and_pad(data, cmd, field_offset):
    new_data = np.array([data[(i * NUM_INP) + field_offset] for i in range(WINDOW_SIZE) if data[(i * NUM_INP) + CMD_OFFSET] == cmd])
    new_data = np.pad(new_data, pad_width=(0, WINDOW_SIZE - len(new_data)))
    new_data = torch.Tensor(np.array([new_data]))
    return new_data

def prepare_data_train(data):

    data_train_cmd= torch.Tensor(np.array([data])).to(device)
    data_train_time = torch.Tensor(np.array([[data[(i * NUM_INP) + TIME_OFFSET] for i in range(WINDOW_SIZE)]])).to(device)
    data_train_freq_lsb = extract_and_pad(data, CMD_LSB, FREQLSB_OFFSET).to(device)
    data_train_freq_msb = extract_and_pad(data, CMD_MSB, FREQMSB_OFFSET).to(device)

    return data_train_cmd, data_train_time, data_train_freq_lsb, data_train_freq_msb


def train():
    
    lr=0.0001

    time_generator = TimeNet().to(device)
    time_optimizer= optim.Adam(time_generator.parameters(), lr=lr, weight_decay=1e-5)
    time_criterion = nn.MSELoss()

    channel_generator = ChannelNet().to(device)
    channel_optimizer= optim.Adam(channel_generator.parameters(), lr=lr, weight_decay=1e-5)
    channel_criterion = nn.CrossEntropyLoss()

    command_generator = CommandNet().to(device)
    command_optimizer= optim.Adam(command_generator.parameters(), lr=lr, weight_decay=1e-5)
    command_criterion = nn.CrossEntropyLoss()

    # The two frequency generators are trained when we are predicting a
    # frequency command only
    freq_msb_generator = FreqNet().to(device)
    freq_msb_optimizer = optim.Adam(command_generator.parameters(), lr=lr, weight_decay=1e-5)
    freq_msb_criterion = nn.MSELoss()

    freq_lsb_generator = FreqNet().to(device)
    freq_lsb_optimizer = optim.Adam(command_generator.parameters(), lr=lr, weight_decay=1e-5)
    freq_lsb_criterion = nn.MSELoss()

    print("Collecting training data")
    train_gen = create_data_split(training_files("../..//training_data/"), 1)
    print("Collected")

    for iteration in range(EPOCHS):
        print(f"Round {iteration}")

        for i in range(ROUND_SZ):
          data = next(train_gen)

          time_optimizer.zero_grad()
          command_optimizer.zero_grad()
          freq_lsb_optimizer.zero_grad()
          freq_msb_optimizer.zero_grad()

          data_train_cmd, data_train_time, data_train_freq_lsb, data_train_freq_msb = prepare_data_train(data['X'][0])

          data_test = data['Y'][0]
          data_test_time = torch.Tensor(np.array([[data_test[0]]])).to(torch.float)
          data_test_channel = torch.Tensor(np.array([data_test[1:3]]))
          data_test_command= torch.Tensor(np.array([onehot_cmd(data_test)]))

          data_test_freq_lsb = torch.Tensor(np.array([[data_test[FREQLSB_OFFSET]]]))
          data_test_freq_msb = torch.Tensor(np.array([[data_test[FREQMSB_OFFSET]]]))

          count = 0
          for x in data_test_command[0]:
              if x.item() == 1.:
                  count += 1
          if count > 1:
              raise "count?!?!?"

          if CUDA:
              data_test_time = data_test_time.cuda()
              data_test_channel = data_test_channel.cuda()
              data_test_command = data_test_command.cuda()
              data_test_freq_lsb = data_test_freq_lsb.cuda()
              data_test_freq_msb = data_test_freq_msb.cuda()

          prediction_time = time_generator(data_train_time)

          #print("TRAIN", data_train)
          #print("TEST", prediction_time, data_test_time)
          time_loss = time_criterion(prediction_time, data_test_time)
          time_loss.backward()
          time_optimizer.step()

          time_ltgt = data_test_time
          time_lpred = prediction_time

          prediction_command = command_generator(data_train_cmd)
          #print(prediction_command.flatten(), data_test_command.flatten())
          command_loss = command_criterion(prediction_command, data_test_command)
          command_loss.backward()
          command_optimizer.step()

          command_ltgt = data_test_command
          command_lpred = prediction_command

          if data_test[CMD_LSB] == 1.:
              #print("Train LSB")
              prediction_freq_lsb = freq_lsb_generator(data_train_freq_lsb)
              freq_lsb_loss = freq_lsb_criterion(prediction_freq_lsb, data_test_freq_lsb)
              freq_lsb_loss.backward()
              freq_lsb_optimizer.step()

          if data_test[CMD_MSB] == 1.:
              #print("Train MSB")
              prediction_freq_msb = freq_msb_generator(data_train_freq_msb)
              freq_msb_loss = freq_msb_criterion(prediction_freq_msb, data_test_freq_msb)
              freq_msb_loss.backward()
              freq_msb_optimizer.step()

        print("Time batch loss:", time_loss.item())
        print("Last prediction:", time_ltgt, time_lpred)
        print("Command batch loss:", command_loss.item())
        print("Last prediction:", command_ltgt, command_lpred)
        print("Freq LSB loss:", freq_lsb_loss.item())
        print("Freq LSB last pred:", prediction_freq_lsb, data_test_freq_lsb)
        print("Freq MSB loss:", freq_msb_loss.item())
        print("Freq MSB last pred:", prediction_freq_msb, data_test_freq_msb)

    return data['X'][0], time_generator.eval(), channel_generator.eval(), command_generator.eval(), freq_lsb_generator.eval(), freq_msb_generator.eval()

seed, time_generator, channel_generator, command_generator, freq_lsb_generator, freq_msb_generator = train()

def pred_cmd(offset, cmd_pred):
    return offset == cmd_pred.argmax().item()

for i in range(10000):

    data_train_cmd, data_train_time, data_train_freq_lsb, data_train_freq_msb = prepare_data_train(seed)

    next_channel = channel_generator(data_train_cmd)

    if pred_cmd(CH_1_OFFSET - 1, next_channel):
        next_channel = 1
    elif pred_cmd(CH_2_OFFSET - 1, next_channel):
        next_channel = 2

    next_time = unnorm(time_generator(data_train_time)[0].item(), NORMALIZE_TIME_BY)
    next_cmd = command_generator(data_train_cmd)

    #print(next_time)
    #print(next_cmd)

    print("Next cmd", next_cmd)

    if pred_cmd(NOP_CMD_OFFSET - 3, next_cmd):
        fresh = nop()
        print("NOP - WHY DID I PREDICT A NO-OP?")
    elif pred_cmd(DUTY_LL_CMD_OFFSET - 3, next_cmd):
        fresh = duty_ll(1, [0, 0, 0, 0, 0, 0, 0], next_time)
        print("DUTY LL TODO")
    elif pred_cmd(VOLENVPER_CMD_OFFSET - 3, next_cmd):
        fresh = volenvper(1, [0, 0, 0, 0, 0, 0, 0], next_time)
        print("VOLENVPER TODO")
    elif pred_cmd(FREQMSB_CMD_OFFSET - 3, next_cmd):
        pred = freq_msb_generator(data_train_freq_msb)[0].item()
        fresh = freqmsb(1, [0, 0, 0, unnorm(pred, 7.), 0, 0], next_time)
        print("FREQMSB AT ", next_time, unnorm(pred, 7.))
    elif pred_cmd(FREQLSB_CMD_OFFSET - 3, next_cmd):
        pred = freq_lsb_generator(data_train_freq_msb)[0].item()
        fresh = freqlsb(1, [0, 0, 0, unnorm(pred, 255.), 0, 0], next_time)
        print(f"FREQLSB {unnorm(pred, 255.)} AT {next_time}")
    else:
        print("??")

    seed = np.concatenate((seed[NUM_INP:], fresh))
    #print("new seed", seed)
    #print(fresh)


Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Collected
Round 0


IndexError: index 1036 is out of bounds for axis 0 with size 1024